# Start

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import akshare as ak  # https://github.com/akfamily/akshare
pd.set_option('display.max_rows', 500)

%matplotlib inline

In [8]:
def _parse_quarter(s):
    x = s.dayofyear
    if x <= 90:
        q = 'Q1'
    elif x <= 181:
        q = 'Q2'
    elif x <= 273:
        q = 'Q3'
    else:
        q = 'Q4'
    return str(s.year) + q
def get_quarter(df, date_column='date'):
    df['DATE'] = pd.to_datetime(df[date_column])
    df['dayofyear'] = df['DATE'].dt.dayofyear
    df.loc[df['DATE'].dt.is_leap_year, 'dayofyear'] -= 1
    df['year'] = df['DATE'].dt.year
    df['quarter'] = df.apply(_parse_quarter, axis=1)
    df = df.set_index('quarter')
    df = df.sort_index()
    return df

def pick_stocks(df, cash_to_invest):
    '''Pick stocks based on momentum and pb, used in the magic formula
    - Look for the TOP_BY_MMT fraction of companies that have the largest MMT_VAR
    - Sort these companies by price-to-book value in ascending order 
        and take the top N_STOCKS stocks
    Returns
        A df with all the stocks chosen and their MMT_VAR and PB
    Note:
        MMT_VAR is a momentum variable like 6 month price momentum
    '''

    cash_for_each_stock = cash_to_invest / N_STOCKS
    top_by_mmt = df[
            (~df['book_value_per_share'].isnull()) & 
            (df['price_to_book_ratio'] > 0)
        ].sort_values(MMT_VAR, ascending=False).iloc[:round(len(df) * TOP_BY_MMT), :]
    stocks_by_mmt_pb = top_by_mmt.sort_values('price_to_book_ratio').head(N_STOCKS).copy()
    stocks_by_mmt_pb['shares_bought'] = cash_for_each_stock / stocks_by_mmt_pb['stock_price']
    stocks_by_mmt_pb_simple = stocks_by_mmt_pb[['stock', 'shares_bought', 'stock_price', 
                                                MMT_VAR, 'price_to_book_ratio']]
    stocks_by_mmt_pb_simple = stocks_by_mmt_pb_simple\
        .rename({'stock_price': 'stock_price_bought',
                 MMT_VAR: MMT_VAR + '_bought',
                 'price_to_book_ratio': 'price_to_book_ratio_bought'}, axis=1)
    return stocks_by_mmt_pb_simple


## Parameters

In [20]:
N_STOCKS = 40
TOTAL_CASH = 100000.0
YEAR_BUY = 2023

# When to buy and sell
MONTH_ACT = 2  # Starting on Jan is a little worse than Feb, but still better than SPY only
QUARTER_ACT = f'Q{MONTH_ACT // 3 + 1}'  # Dummy variable. If acting on Feb, the quarter is Q1
QUARTER_BV = f'Q{MONTH_ACT // 3 + 3}'  # If acting in Q1, I get the book value of previous Q3 for most stocks to compute PB
MONTH_PREV = MONTH_ACT - 6 if MONTH_ACT - 6 > 0 else MONTH_ACT + 6

N_TOP_BY_MKT_CAP = 300  # Choose from the top N of sp500
TOP_BY_MMT = 0.2  # The top fraction of stocks ranked by MMT
MMT_VAR = 'stock_price_mmt_6m'
TAX_FACTOR = (1 - 0.1)

# Download SP500 component history
Downloading the top 300 takes about 2 hours

## Raw data

In [24]:
## Get fundamental data (seasonal) or price data (daily) 

is_from_scratch = False  # If no stock data has been downloaded 
to_download = 'fundamental'  # download fundamental or price data

# Get stock list (ordered by capital)

df_sp500_list = pd.read_excel('sp500_fulllist_ranked.xlsx', engine='openpyxl', sheet_name=str(YEAR_BUY))
df_stocks = df_sp500_list.stock
stocks = list(df_stocks[df_stocks != 'GOOG'].values)  # There is GOOGL already
stocks += ['OHI']  # I like OHI

file_name = {
    'fundamental': f'sp500_history_raw_{str(YEAR_BUY)}.csv',
    'price': f'sp500_history_price_raw_{str(YEAR_BUY)}.csv',
}
min_row = {
    'fundamental': 3,
    'price': 190,
}
anom = {'Failed': [], 'Short': []}

if not is_from_scratch:  # 
    df_stock_all = read_csv(file_name[to_download])

try:
    # If df_stock_all is defined, take stocks downloaded
    stock_downloaded = df_stock_all.stock.unique()
    print(f'Downloaded {len(stock_downloaded)}, {stock_downloaded}')
except NameError: 
    df_stock_all = pd.DataFrame()

for stock_symbol in stocks:
    try:
        if stock_symbol in stock_downloaded:
            continue
    except NameError: 
        pass
    try:
        if to_download == 'fundamental':
            df_pe = ak.stock_us_fundamental(stock=stock_symbol, symbol="PE")
            df_pb = ak.stock_us_fundamental(stock=stock_symbol, symbol="PB")
            df_stock = pd.merge(df_pe, df_pb.drop('stock_price', axis=1), on='date')
        elif to_download == 'price': 
            df_stock = ak.stock_us_daily(symbol=stock_symbol)
        else:
            print('Wrong variable name')
    except IndexError:
        print(f'Failed for {stock_symbol}')
        anom['Failed'].append(stock_symbol)
        continue        
    df_stock['stock'] = stock_symbol
    df_stock_all = df_stock_all.append(df_stock)
    print(f"{(stock_symbol, df_stock.date.min(), df_stock.date.max(), len(df_stock))}")
    if len(df_stock) < min_row[to_download]:
        anom['Short'].append(stock_symbol)
    df_stock_all.to_csv(file_name[to_download], index=False)    

Downloaded 25, ['AAPL' 'MSFT' 'AMZN' 'GOOGL' 'BRK.B' 'NVDA' 'XOM' 'UNH' 'JNJ' 'JPM'
 'TSLA' 'V' 'PG' 'HD' 'MA' 'META' 'CVX' 'MRK' 'LLY' 'ABBV' 'PFE' 'BAC'
 'AVGO' 'KO' 'PEP']
('TMO', '2009-12-31', '2023-01-25', 53)
('COST', '2009-11-30', '2023-01-25', 54)
('WMT', '2009-07-31', '2023-01-25', 54)
('MCD', '2009-12-31', '2023-01-25', 53)
('CSCO', '2009-07-31', '2023-01-25', 54)
('ABT', '2009-12-31', '2023-01-25', 53)
('DIS', '2009-12-31', '2023-01-25', 53)
('DHR', '2009-12-31', '2023-01-25', 53)
('ACN', '2009-11-30', '2023-01-25', 54)
('CMCSA', '2009-12-31', '2023-01-25', 53)
('VZ', '2009-12-31', '2023-01-25', 53)
('WFC', '2009-09-30', '2023-01-25', 53)
('ADBE', '2009-11-30', '2023-01-25', 54)
('NEE', '2009-12-31', '2023-01-25', 53)
('LIN', '2009-12-31', '2023-01-25', 53)
('NFLX', '2009-12-31', '2023-01-25', 53)


URLError: <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>

## Processed

In [117]:
df_stock_all = pd.read_csv('sp500_history_raw.csv')

MIN_QUARTERS = 9  # Remove stocks with fewer than some quarters
MAX_QUARTERS_NEG_PB = 20  # Remove stocks with more than some quarters with negatives pb

df_stock_sub = df_stock_all[df_stock_all.stock.isin((df_stock_all.groupby('stock').size() >= 
                                                     MIN_QUARTERS).index.values)].copy()
df_stock_sub = get_quarter(df_stock_sub)
df_stock_sub = df_stock_sub.drop(['DATE', 'dayofyear', ], axis=1).sort_values(['date', 'stock'])
# Convert $dollar to float
df_stock_sub['ttm_net_eps'] = df_stock_sub.ttm_net_eps.str[1:].replace('', np.nan).astype(float)
df_stock_sub['book_value_per_share'] = df_stock_sub.book_value_per_share.str[1:].replace('', np.nan).astype(float)
# Replace inf pe or pb to 0
df_stock_sub = df_stock_sub.replace(np.inf, 0)
# Remove stocks with over XX quarters with neg equity (pb)
df_stock_sub_neg = df_stock_sub[df_stock_sub.price_to_book_ratio < 0].groupby('stock').size()
stocks_sub_neg = df_stock_sub_neg[df_stock_sub_neg >= MAX_QUARTERS_NEG_PB].index.values
df_stock_sub = df_stock_sub[~df_stock_sub.stock.isin(stocks_sub_neg)]
df_stock_sub.to_csv('sp500_history_filterd.csv')

# Backtest the magic-formula with actual daily price

Here I get daily price history data and use the price of the action day (like Feb 1st) to compute price momentum. Also I use a latest full list of SP500, but take only the first N stocks

Method: 
- Select the N_TOP_BY_MKT_CAP top stocks from SP500
- Starting from 2010, take actions on the first trading day of Feb(MONTH_ACT) of each year
- First choose the top 20% (TOP_BY_MMT = 0.2) of stocks ranked by 6 month price momentum 
(price of first day of MONTH_ACT minus price of first day of previous MONTH_PREV)
- Then choose the top N_STOCKS ranked by PB (price of first day of MONTH_ACT / book value of previous Q3)
- On the first trading day of MONTH_ACT of each year, sell all stocks from the previous year with tax rate of 90% (TAX_FACTOR = (1 - 0.1)), and buy new stocks with the money by repeating the previous two steps  

Results:
- Act on the first trading day of Feb, using price of that day and BV of Q3
- Choose the top 40 gives good return
- Each year, the gain (after 10% lt tax) is better than SPY

Caveat: 
- If stock splits, the price momentum is unreasonable and the stock is skipped for that year
- If using finviz, the BV is updated with Q4 results for some stocks

## Get momentum

In [172]:
df_p_history = pd.read_csv('sp500_history_price_raw.csv')
df_p_history['date'] = pd.to_datetime(df_p_history['date'])

# Remove stocks that didn't last until the recent
dt_cutoff = pd.to_datetime('2022-01-01')
df_p_history['max_date'] = df_p_history.groupby('stock')['date'].transform(max)
df_p_history = (df_p_history[df_p_history['max_date'] >= dt_cutoff]).drop('max_date', axis=1)

In [173]:
df_p_history = df_p_history.sort_values(['stock', 'date'])
df_p_history['year'] = df_p_history.date.dt.year
df_p_history['month'] = df_p_history.date.dt.month
df_p_history['day1_of_month'] = df_p_history.groupby(['stock', 'year', 'month'])['date'].transform(min)# Get momentum

# Get the first day of MONTH_PREV and of MONTH_ACT, compute the momentum
df_p_history_prev = df_p_history[(df_p_history.month == MONTH_PREV) &
                                 (df_p_history.date == df_p_history.day1_of_month)]
df_p_history_curr = df_p_history[(df_p_history.month == MONTH_ACT) &
                                 (df_p_history.date == df_p_history.day1_of_month)]
df_p_history_curr['year_prev'] = df_p_history_curr.year - 1
df_p_history_prev['year_prev'] = df_p_history_prev.year
cols = ['date', 'close', 'stock', 'year_prev']
df_p_history_mmt = pd.merge(df_p_history_prev[cols], 
                            df_p_history_curr[cols + ['year']],
                            on=['stock', 'year_prev'], 
                            suffixes=['_prev', '']
                           )

In [219]:
df_p_history_mmt['stock_price_mmt_6m'] = df_p_history_mmt['close'] / df_p_history_mmt['close_prev'] - 1

## Get PB

In [175]:
df_pb_history = pd.read_csv('sp500_history_filterd.csv', index_col=0)

In [176]:
df_pb_quarter = df_pb_history[df_pb_history.index.str.endswith(QUARTER_BV)]\
    [['date', 'stock_price', 'book_value_per_share', 'stock', 'year']]\
    .reset_index(drop=True).rename({'year': 'year_prev', 'stock_price': 'stock_price_pb'}, axis=1)

## Pick stocks and backtest

In [220]:
df_p_pb = pd.merge(df_p_history_mmt[['stock', 'year_prev', 'date', 'close', 
                                     'year', 'stock_price_mmt_6m']], 
                   df_pb_quarter, 
                   on=['stock', 'year_prev'], suffixes=['', '_pb'])\
            .rename({'close': 'stock_price'}, axis=1)\
            .drop(['year_prev'], axis=1)

df_p_pb['price_to_book_ratio'] = df_p_pb['stock_price'] / df_p_pb['book_value_per_share']

In [221]:
# Choose only the top N companies of SP500 to start with
df_rank = pd.read_csv('sp500_fulllist_ranked.csv')
top_stocks = df_rank[df_rank['rank'] <= N_TOP_BY_MKT_CAP].stock.values
df_p_pb = df_p_pb[df_p_pb.stock.isin(top_stocks)]

## Get the stocks based on the magic formula - xxxx date

In [232]:
stocks_split = ['GE']
df_row = df_p_pb[df_p_pb.year == YEAR_BUY]
df_row = df_row[~df_row.stock.isin(stocks_split)]
stocks_invested = pick_stocks(df_row, cash_to_invest=TOTAL_CASH)
stocks_invested['shares_bought'] = stocks_invested['shares_bought'].round()
stocks_invested['stock_price_mmt_6m_bought'] = (stocks_invested['stock_price_mmt_6m_bought'] * 100).round()

In [233]:
# stocks_invested.sort_values('stock_price_mmt_6m_bought', ascending=False)
cols = ['stock', 'price_to_book_ratio_bought', 'stock_price_mmt_6m_bought']
stocks_invested.sort_values('price_to_book_ratio_bought').reset_index(drop=True)

,stock,shares_bought,stock_price_bought,stock_price_mmt_6m_bought,price_to_book_ratio_bought
0,AIG,42.0,59.11,25.0,0.752419
1,MET,37.0,68.09,18.0,0.829354
2,BK,41.0,60.69,18.0,1.142292
3,WFC,45.0,55.60,21.0,1.163180
4,TFC,39.0,64.02,18.0,1.240457
5,AFL,40.0,62.89,15.0,1.242394
6,MPC,34.0,74.02,36.0,1.316379
7,ED,29.0,86.05,16.0,1.397823
8,FITB,54.0,45.94,27.0,1.407044
9,BAC,53.0,46.94,24.0,1.419843


In [ ]:
# https://finviz.com/screener.ashx?v=152&f=idx_sp500,ta_perf_26w10o&ft=3&o=pb